In [ ]:
import random
import numpy as np
from collections import deque
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

## Group Definition:
Let $A = \begin{bmatrix}1 & 2 \\ 0 & 1\end{bmatrix}$, $B = \begin{bmatrix}1 & 0 \\ 2 & 1\end{bmatrix}$.
The group $\Gamma \coloneqq \langle A,B\rangle \subseteq SL_2(\mathbb{Z})$ is an index $12$ subgroup. The diagonal entries are congruent to $1\pmod{4}$ and the non-diagonal entries and divisible by $2$. 

We define $C = A^{-1}$ and $D = B^{-1}$.

We can generate with any coset by starting at a representative from each coset and see if we get our way back to it?

In [2]:
def matrix_to_tuple(matrix: np.array) -> tuple:
    return (matrix[0][0], matrix[0][1], 
            matrix[1][0], matrix[1][1]) 

def tuple_to_matrix(tuple: tuple) -> np.array:
    return np.array([[tuple[0], tuple[1]], [tuple[2], tuple[3]]])

In [ ]:
# index 12 according to alex's paper. Is it congruent to identity mod 2 or mod 4?
# can generate with any coset I want by starting at a representative from each coset and see if we get our way back to it
A = np.array([[1, 3], [0, 1]])
B = np.array([[1, 0], [3, 1]])

# elements on the diagonal are 1 mod 4. 
# elements not on the diagonal are 0 mod 2. 

# C is the inverse of A
# D is the inverse of B
C = np.linalg.inv(A)
D = np.linalg.inv(B)

identity = np.array([[1, 0], [0, 1]])

In [ ]:
def epsilon_greedy_search(Epsilon: float, qtable: dict, state: np.array) -> int:
    '''
    Makes a random step with probability Epsilon, and otherwise makes the best move from the Q_table.
    (exploration vs exploitation)
    '''
    if (random.random() < Epsilon):
        # 0 is 'apply matrix A', 1 is 'apply matrix B'
        # 2 is 'apply matrix C', 3 is 'apply matrix D'
        return random.choice([0, 1, 2, 3])
    else:
        # get the best move for the current state
        return best_move_for_a_state(Q_table=qtable, state=state)
    
# I would like to return the best move for a given state
def best_move_for_a_state(Q_table, state):
    # vals = Q_table[(state[0][1], state[0][2], state[1][2])]

    apply_A = state @ A
    apply_B = state @ B
    apply_C = state @ C
    apply_D = state @ D

    vals = [0, 0, 0, 0]
    vals[0] = Q_table[matrix_to_tuple(apply_A)]
    vals[1] = Q_table[matrix_to_tuple(apply_B)]
    vals[2] = Q_table[matrix_to_tuple(apply_C)]
    vals[3] = Q_table[matrix_to_tuple(apply_D)]

    # if we haven't visited this state before, return a random choice of 0, 1, 2, or 3
    if vals==[0, 0, 0, 0]:
        return random.choice([0, 1, 2, 3])
    
    # if we have visited this state before, return the current best choice
    return np.argmax(vals)

# over a given state, return the maximum value of the table for that state
def max_a_prime(Q_table, state):
    apply_A = state @ A
    apply_B = state @ B
    apply_C = state @ C
    apply_D = state @ D

    vals = [0, 0, 0, 0]
    vals[0] = Q_table[matrix_to_tuple(apply_A)]
    vals[1] = Q_table[matrix_to_tuple(apply_B)]
    vals[2] = Q_table[matrix_to_tuple(apply_C)]
    vals[3] = Q_table[matrix_to_tuple(apply_D)]
    
    return max(vals)

In [ ]:
MAX_REWARD = 100
STEP_PENALTY = -1

def getReward(matrix: np.array) -> int:
    if (matrix == identity).all():
        return MAX_REWARD
    else:
        return STEP_PENALTY

In [ ]:
df = pd.read_csv("../Data_Generation/Data_files/subset_sl2_Z_3s.csv")

In [ ]:
def get_next_step(oldObs: np.array, action: int) -> tuple[np.array, int, bool]:
    '''
    Apply matrix multiplication to take a step and get associated reward

    returns: 3-tuple of the (step taken, associated reward of step, true if at identity)
    '''

    assert(action in [0, 1, 2, 3])

    next_state = []
    if action==0:
        next_state = oldObs @ A
    elif action==1:
        next_state = oldObs @ B
    elif action==2:
        next_state = oldObs @ C
    else:
        next_state = oldObs @ D
    curReward = getReward(next_state)
    done = (curReward == MAX_REWARD)
    return (next_state, curReward, done)

In [ ]:
def apply_mat(mat: np.array, index: int) -> np.array:
    if index==0:
        return mat @ A
    elif index==1:
        return mat @ B
    elif index==2:
        return mat @ C
    elif index==3:
        return mat @ D
    raise ValueError("Index is not between 0 and 3")

In [8]:
def tuple_to_matrix(tuple):
    return np.array([[tuple[0], tuple[1]], [tuple[2], tuple[3]]])

In [9]:
df[(df['val1'] % 2 == 1) & (df['val2'] % 2 == 0) & (df['val3'] % 2 == 0) & (df['val4'] % 2 == 1)]

,val1,val2,val3,val4
9,1.0,0.0,6.0,1.0
19,271.0,-96.0,48.0,-17.0
21,213049.0,673746.0,601302.0,1901557.0
22,93637.0,16578.0,16476.0,2917.0
24,37.0,-6.0,-6.0,1.0
...,...,...,...,...
9942,5221.0,-31926.0,14088.0,-86147.0
9944,1095211.0,-117576.0,3106386.0,-333485.0
9961,1763227.0,4690950.0,-285456.0,-759437.0
9962,-647.0,-246.0,192.0,73.0


In [10]:
Verify that the diagonal entries are congruent to $1\pmod{4}$ and the non-diagonal entries and divisible by $2$. 

,val1,val2,val3,val4
9,1.0,0.0,6.0,1.0
19,271.0,-96.0,48.0,-17.0
21,213049.0,673746.0,601302.0,1901557.0
22,93637.0,16578.0,16476.0,2917.0
24,37.0,-6.0,-6.0,1.0
...,...,...,...,...
9942,5221.0,-31926.0,14088.0,-86147.0
9944,1095211.0,-117576.0,3106386.0,-333485.0
9961,1763227.0,4690950.0,-285456.0,-759437.0
9962,-647.0,-246.0,192.0,73.0


In [ ]:
filter_df = df[df['val1'] % 4 == 1]
filter_df = filter_df[filter_df['val2'] % 2 == 0]
filter_df = filter_df[filter_df['val3'] % 2 == 0]
filter_df = filter_df[filter_df['val4'] % 4 == 1]
filter_df

In [11]:
EPISODES = 30000
LEARNING_RATE = .9
DISCOUNT_FACTOR = .99
EPSILON = 1
EPSILON_DECAY = .9999

random.seed(42)

# starts with an estimate of zero reward for each state.
# adapted from ChatGPT
Q_table = defaultdict(lambda: 0)

episode_reward_record = deque(maxlen=100)

for i in range(EPISODES):
    episode_reward = 0
    done = False
    # choose a random starting row
    # adapted from https://stackoverflow.com/questions/15923826/random-row-selection-in-pandas-dataframe
    cur_row = df.sample(1)
    obs = np.array([
        [int(cur_row['val1']), int(cur_row['val2'])], 
        [int(cur_row['val3']), int(cur_row['val4'])]
        ])

    index = 1

    while (not done):
        # perform an epsilon greedy action 
        # Q(s, a) = (1-LEARNING_RATE)Q(s, a) + (LEARNING_RATE)(r + DISCOUNT_FACTOR(max a'(Q(s', a'))))
        action = epsilon_greedy_search(Epsilon=EPSILON, qtable=Q_table, state=obs)

        oldObs = obs
        obs,reward,done = get_next_step(oldObs, action)

        # if done:
        #     assert(1==2)
        
        Q_table[matrix_to_tuple(obs)] = (1-LEARNING_RATE) * Q_table[matrix_to_tuple(obs)] + (LEARNING_RATE) * (reward + DISCOUNT_FACTOR * (max_a_prime(Q_table, obs)))

        episode_reward += reward # update episode reward

        index += 1
        # if we take more than 100 steps, end this iteration early (we are probably not making progress)
        if index > 100:
            done=True

    # decay the epsilon
    EPSILON *= EPSILON_DECAY

    # record the reward for this episode
    episode_reward_record.append(episode_reward) 

    if i%100 ==0 and i>0:
        print("Average reward for the last 100 iterations: " + str(sum(list(episode_reward_record))/100))
        print("epsilon: " + str(EPSILON) )



/var/folders/t9/gxb6q8zj21dff090q066td740000gn/T/ipykernel_5789/1460151504.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  [int(cur_row['val1']), int(cur_row['val2'])],
/var/folders/t9/gxb6q8zj21dff090q066td740000gn/T/ipykernel_5789/1460151504.py:23: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  [int(cur_row['val3']), int(cur_row['val4'])]


Average reward for the last 100 iterations: -96.02
epsilon: 0.989950333757503
Average reward for the last 100 iterations: -98.01
epsilon: 0.9800996732739187
Average reward for the last 100 iterations: -100.0
epsilon: 0.9703470333764725
Average reward for the last 100 iterations: -100.0
epsilon: 0.9606914386955115
Average reward for the last 100 iterations: -100.0
epsilon: 0.9511319235669539
Average reward for the last 100 iterations: -100.0
epsilon: 0.9416675319357145
Average reward for the last 100 iterations: -100.0
epsilon: 0.9322973172600907
Average reward for the last 100 iterations: -98.01
epsilon: 0.9230203424170932
Average reward for the last 100 iterations: -96.03
epsilon: 0.9138356796087268
Average reward for the last 100 iterations: -98.01
epsilon: 0.9047424102692004
Average reward for the last 100 iterations: -100.0
epsilon: 0.89573962497306
Average reward for the last 100 iterations: -96.04
epsilon: 0.8868264233442354
Average reward for the last 100 iterations: -100.0
epsi

In [ ]:
def access_Q_table(mat):
    return Q_table[matrix_to_tuple(mat)]

In [13]:
access_Q_table(np.array([[1, 3], [0, 1]]))

4924.623115577806

In [14]:
access_Q_table(A)

4924.623115577806

In [ ]:
# test with the other dataframe. 
test_df = pd.read_csv("../Data_Generation/Data_files/subset_sl2_Z_3s_test.csv")

In [ ]:
def matrix_to_num_steps(cur_matrix):
    index = 1
    for i in range(50):
        if (cur_matrix==identity).all():
            return i
        outputs = [0] * 4
        outputs[0] = Q_table[matrix_to_tuple(cur_matrix@ A)]
        outputs[1] = Q_table[matrix_to_tuple(cur_matrix@ B)]
        outputs[2] = Q_table[matrix_to_tuple(cur_matrix@ C)]
        outputs[3] = Q_table[matrix_to_tuple(cur_matrix@ D)]
        index = np.argmax(outputs)
        if index==0:
            cur_matrix = cur_matrix @ A
        elif index==1:
            cur_matrix = cur_matrix @ B
        elif index==2:
            cur_matrix = cur_matrix @ C
        elif index==3:
            cur_matrix = cur_matrix @ D
    return 100

In [ ]:
def test_Q_learning(cur_row: np.array) -> int:
    cur_matrix = np.array([
        [int(cur_row['val1']), int(cur_row['val2'])], 
        [int(cur_row['val3']), int(cur_row['val4'])]
        ])
    return matrix_to_num_steps(cur_matrix)

test_df['num_moves_Q_learning_needs'] = test_df.apply(test_Q_learning, axis=1)

In [18]:
print("The proportion of starting positions in the test dataset that we can find a route to the origin that's <50 steps: ")
sum(test_df['num_moves_Q_learning_needs']!=100)/test_df.shape[0]

The proportion of starting positions in the test dataset that we can find a route to the origin that's <50 steps: 


0.2668

All paths from $A$ to $I$ for $A \in \Gamma$ that take less than 20 matrix multiplications

In [ ]:
print("Of these, the proportion of times where we learned a path that was < 20 moves: ")
# encouraging because all of these were generated as sequences of 30 moves
# so we've found significantly faster paths back to the origin for almost all moves that we find a path to the origin 
sum(test_df['num_moves_Q_learning_needs']<20)/sum(test_df['num_moves_Q_learning_needs']!=100)

In [ ]:
test_df

In [ ]:
filtered_df = test_df[test_df['num_moves_Q_learning_needs']!=100]

In [21]:
def first_matrix_to_apply(cur_row):
    outputs = [0, 0, 0, 0]
    cur_matrix = np.array([
        [int(cur_row['val1']), int(cur_row['val2'])], 
        [int(cur_row['val3']), int(cur_row['val4'])]
        ])
    outputs[0] = Q_table[matrix_to_tuple(cur_matrix@ A)]
    outputs[1] = Q_table[matrix_to_tuple(cur_matrix@ B)]
    outputs[2] = Q_table[matrix_to_tuple(cur_matrix@ C)]
    outputs[3] = Q_table[matrix_to_tuple(cur_matrix@ D)]
    return np.argmax(outputs)

filtered_df['first_move_by_Q_learning'] = filtered_df.apply(first_matrix_to_apply, axis=1)

/var/folders/t9/gxb6q8zj21dff090q066td740000gn/T/ipykernel_5789/2721632581.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['first_move_by_Q_learning'] = filtered_df.apply(first_matrix_to_apply, axis=1)


In [22]:
filtered_df

,val1,val2,val3,val4,num_moves_Q_learning_needs,first_move_by_Q_learning
4,28945.0,-8727.0,10713.0,-3230.0,10,1
7,-233.0,1485.0,1323.0,-8432.0,10,0
9,1.0,0.0,6.0,1.0,2,3
10,-1619.0,9231.0,-600.0,3421.0,9,0
11,217.0,-75.0,-1383.0,478.0,8,1
...,...,...,...,...,...,...
9980,478.0,-75.0,2919.0,-458.0,9,1
9986,-152.0,483.0,45.0,-143.0,6,0
9989,55.0,-6.0,-9.0,1.0,5,1
9991,415.0,-1293.0,147.0,-458.0,8,0


In [ ]:
bound = int(filtered_df.shape[0] * 0.6)
plus_one = bound+1
train = filtered_df.iloc[1:bound]
test = filtered_df.iloc[plus_one:filtered_df.shape[0]]

In [ ]:
def get_Q_value(row):
    return Q_table[(int(row['val1']), 
    int(row['val2']), 
    int(row['val3']),
    int(row['val4'])
    )]

In [ ]:
train.to_csv("../Data_Generation/Data_files/subset3_train_rows_SL2Z_Q_learn.csv", index=False)
test.to_csv("../Data_Generation/Data_files/subset3_test_rows_SL2Z_Q_learn.csv", index=False)

In [26]:
def mod_2_is_identity(test_tuple):
    assert len(test_tuple)==4
    return (test_tuple[0] % 2 == 1 and 
            test_tuple[1] % 2 == 0 and 
            test_tuple[2] % 2 == 0 and 
            test_tuple[3] % 2 == 1)